In [1]:
%load_ext autoreload
%autoreload 2

import os
import pickle

import numpy as np

from utils import init_folder, load_list_of_elements, get_dataset_and_metadata, get_dataset_from_file

import time
import optuna
import argparse


from gluonts.evaluation import make_evaluation_predictions
from gluonts.mx.trainer import Trainer
from gluonts.mx.trainer.callback import TrainingHistory
from gluonts.mx.distribution import StudentTOutput, MultivariateGaussianOutput
from sklearn.metrics import mean_absolute_error
from gluonts.dataset.multivariate_grouper import MultivariateGrouper

from gluonts.mx.model.simple_feedforward import SimpleFeedForwardEstimator
from my_models.gluonts_models.univariate.feedforward_point import SimpleFeedForwardEstimator as FF_gluonts_univariate_point
from my_models.gluonts_models.ffn_multivar import SimpleFeedForwardEstimator as FF_gluonts_multivariate
from gluonts.mx.model.transformer import TransformerEstimator
from gluonts.mx.model.deepar import DeepAREstimator 

import matplotlib.pyplot as plt
import json
from pathlib import Path
from gluonts.dataset.split import split
from gluonts.dataset.common import ListDataset

from data_manager import GluonTSDataManager


/opt/conda/envs/clts2/lib/python3.10/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/opt/conda/envs/clts2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

seasonality = {
    'nn5_weekly': 52.17857142857143,
    'us_births_dataset': 7,
    'weather': 7,
    'sunspot_without_missing': 7,
    'solar_10_minutes': 144,
    'hospital': 12,
    'rideshare_without_missing': 24,
    'fred_md': 12
}
class Objective:

    def __init__( self, MODEL, DATASET_NAME, ctx, DATASET_FILE_FOLDER, multivariate=False):
        
        data_manager = GluonTSDataManager(DATASET_NAME, multivariate, DATASET_FILE_FOLDER)
        self.n_features = data_manager.n_features
        self.context_length = data_manager.context_length
        self.prediction_length = data_manager.prediction_length
        self.train_original = data_manager.train_dataset
        self.test = data_manager.test_dataset
        self.freq = data_manager.freq
        self.seasonality = seasonality[DATASET_NAME]
        self.dataset_name = DATASET_NAME

        self.model = MODEL
        self.multivariate = multivariate
        self.ctx = ctx

        self.train, test_template = split(self.train_original, offset=-self.prediction_length)
        validation = test_template.generate_instances(
            prediction_length=self.prediction_length,
        )
        # Assuming `validation` is a list of (input, output) pairs
        validation_data = [
            {
                "start": v[0]["start"],  # replace with the actual start time
                "target": np.concatenate([v[0]['target'], v[1]['target']]),
            }
            for v in validation
        ]

        self.validation = ListDataset(validation_data, freq=self.freq)

        print(self.model, self.multivariate, self.ctx)


    def get_params(self, trial) -> dict:

        if self.model == 'feedforward':
          return {
              "num_hidden_dimensions": [trial.suggest_int("hidden_dim_{}".format(i), 10, 100) for i in range(trial.suggest_int("num_layers", 1, 5))],
              "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-6, 1e-4),
              "trainer:epochs": trial.suggest_int("trainer:epochs", 10, 100),
          }
        elif self.model == 'deepar':
           return {
              "num_cells": trial.suggest_int("num_cells", 10, 100),
              "num_layers": trial.suggest_int("num_layers", 1, 5),
              "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-6, 1e-4),
              "trainer:epochs": trial.suggest_int("trainer:epochs", 10, 100)
           }
        elif self.model == 'transformer':
          # num_heads must divide model_dim
          valid_pairs = [ (i,d) for i in range(10,101) for d in range(1,11) if i%d == 0  ]
          model_dim_num_heads_pair = trial.suggest_categorical("model_dim_num_heads_pair", valid_pairs)

          return {
              "inner_ff_dim_scale": trial.suggest_int("inner_ff_dim_scale", 1, 5),
              "model_dim": model_dim_num_heads_pair[0],
              "embedding_dimension": trial.suggest_int("embedding_dimension", 1, 10),
              "num_heads": model_dim_num_heads_pair[1],
              "dropout_rate": trial.suggest_uniform("dropout_rate", 0.0, 0.5),
              "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-6, 1e-4),
              "trainer:epochs": trial.suggest_int("trainer:epochs", 10, 100),
          }


    def __call__(self, trial):

        params = self.get_params(trial)

        return self.train_and_test(params)

    def train_and_test(self, params, save=False):

      history = TrainingHistory()
      if self.model == 'feedforward' and not self.multivariate:
        # estimator = SimpleFeedForwardEstimator_nonorm_point(
        #     num_hidden_dimensions= params['num_hidden_dimensions'], #num_hidden_dimensions,
        #     prediction_length=self.prediction_length,
        #     context_length=self.context_length,
        #     n_features=self.n_features,
        #     trainer=Trainer(ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'], num_batches_per_epoch=100),
        # )
        estimator = FF_gluonts_univariate_point(
            num_hidden_dimensions= params['num_hidden_dimensions'], #num_hidden_dimensions,
            prediction_length=self.prediction_length,
            context_length=self.context_length,
            batch_normalization=True,
            mean_scaling=False,
            trainer=Trainer(ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                             num_batches_per_epoch=100, callbacks=[history]),
        )
      elif self.model == 'feedforward' and self.multivariate:
        estimator = FF_gluonts_multivariate(
            num_hidden_dimensions= params['num_hidden_dimensions'], #num_hidden_dimensions,
            prediction_length=self.prediction_length,
            context_length=self.context_length,
            mean_scaling=False,
            # batch_normalization=True,
            distr_output=MultivariateGaussianOutput(dim=self.n_features),
            trainer=Trainer(ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                             num_batches_per_epoch=100, callbacks=[history]),
        )
      elif self.model == 'deepar':
        estimator = DeepAREstimator(
            freq=self.freq,
            context_length=self.context_length,
            distr_output=StudentTOutput(),
            prediction_length=self.prediction_length,
            num_cells= params['num_cells'],
            num_layers= params['num_layers'],
            trainer=Trainer(ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                             num_batches_per_epoch=100, callbacks=[history]),
        )
      elif self.model == 'transformer':
        estimator = TransformerEstimator(
            freq=self.freq,
            context_length=self.context_length,
            prediction_length=self.prediction_length,
            inner_ff_dim_scale= params['inner_ff_dim_scale'],
            model_dim= params['model_dim'],
            embedding_dimension= params['embedding_dimension'],
            num_heads= params['num_heads'],
            dropout_rate= params['dropout_rate'],
            trainer=Trainer(ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                             num_batches_per_epoch=100, callbacks=[history]),
        )

      ## TRAIN
      predictor = estimator.train(self.train, self.validation)
      ## EVALUATE
      if not save:
         test = self.validation
      else:
         test = self.test
      forecast_it, ts_it = make_evaluation_predictions(
          dataset=test,  # validation dataset
          predictor=predictor,  # predictor
          num_samples=100,  # number of sample paths we want for evaluation
      )

      forecasts = list(forecast_it)

      final_forecasts = []
      for f in forecasts:
          final_forecasts.append(f.median)

      mase_metrics = []
      for item_id, ts in enumerate(test):
        training_data = ts["target"].T[:-self.prediction_length]
        ground_truth = ts["target"].T[-self.prediction_length:]

        y_pred_naive = np.array(training_data)[:-int(self.seasonality)]
        mae_naive = mean_absolute_error(np.array(training_data)[int(self.seasonality):], y_pred_naive, multioutput="uniform_average")

        mae_score = mean_absolute_error(
            np.array(ground_truth),
            final_forecasts[item_id],
            sample_weight=None,
            multioutput="uniform_average",
        )

        epsilon = np.finfo(np.float64).eps
        if mae_naive == 0:
          continue
        mase_score = mae_score / np.maximum(mae_naive, epsilon)


        mase_metrics.append(mase_score)
      if not save:
        return np.mean(mase_metrics)
      
      # make directory called saved_nonorm_{self.model}_{self.dataset_name}
      dir_name = f'saved_nonorm_{self.model}_{self.dataset_name}'
      os.makedirs(dir_name, exist_ok=True)

      return np.mean(mase_metrics), predictor, dir_name, history

In [3]:
DATASET_NAME, model_choice, ctx, DATASET_FILE_FOLDER, n_trials = 'nn5_weekly', 'feedforward', 'gpu', None, 1
multivariate = False
study = optuna.create_study(direction="minimize")
obj = Objective(
        model_choice,DATASET_NAME, ctx, DATASET_FILE_FOLDER, multivariate
    )
study.optimize(
    obj,
    n_trials=n_trials,
)

[I 2024-02-10 14:24:25,913] A new study created in memory with name: no-name-56e9b8f5-24f7-40f5-9552-e8ffd39d0c65


/tmp/ipykernel_16320/473181749.py:52: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-6, 1e-4),
[14:24:25] /work/mxnet/src/base.cc:79: cuDNN lib mismatch: linked-against version 8101 != compiled-against version 8500.  Set MXNET_CUDNN_LIB_CHECKING=0 to quiet this warning.


feedforward False gpu


100%|██████████| 100/100 [00:03<00:00, 26.15it/s, epoch=1/75, avg_epoch_loss=1.4]
4it [00:00, 104.74it/s, epoch=1/75, validation_avg_epoch_loss=1.37]
100%|██████████| 100/100 [00:03<00:00, 28.05it/s, epoch=2/75, avg_epoch_loss=1.38]
4it [00:00, 348.59it/s, epoch=2/75, validation_avg_epoch_loss=1.36]
100%|██████████| 100/100 [00:04<00:00, 22.04it/s, epoch=3/75, avg_epoch_loss=1.38]
4it [00:00, 49.79it/s, epoch=3/75, validation_avg_epoch_loss=1.38]
100%|██████████| 100/100 [00:08<00:00, 12.46it/s, epoch=4/75, avg_epoch_loss=1.37]
4it [00:00, 57.57it/s, epoch=4/75, validation_avg_epoch_loss=1.38]
100%|██████████| 100/100 [00:09<00:00, 10.83it/s, epoch=5/75, avg_epoch_loss=1.37]
4it [00:00, 43.33it/s, epoch=5/75, validation_avg_epoch_loss=1.39]
100%|██████████| 100/100 [00:09<00:00, 10.81it/s, epoch=6/75, avg_epoch_loss=1.36]
4it [00:00, 40.67it/s, epoch=6/75, validation_avg_epoch_loss=1.34]
100%|██████████| 100/100 [00:04<00:00, 22.18it/s, epoch=7/75, avg_epoch_loss=1.34]
4it [00:00, 30.6

In [1]:
# Python
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.mx.model.simple_feedforward import SimpleFeedForwardEstimator
from my_models.gluonts_models.univariate.feedforward_point import SimpleFeedForwardEstimator as FF_gluonts_univariate_point
from gluonts.mx.trainer import Trainer
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions

# Load the "m4_hourly" dataset
dataset = get_dataset("m4_hourly")

# Define a trainer
trainer = Trainer(epochs=1, ctx="gpu")

# Create an estimator
estimator = FF_gluonts_univariate_point(
    num_hidden_dimensions=[10],
    prediction_length=dataset.metadata.prediction_length,
    context_length=100,
    trainer=trainer,
)

# Train the model
predictor = estimator.train(dataset.train)

# Make prediction
forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_samples=100
)

# Evaluate the model
evaluator = Evaluator()
agg_metrics, item_metrics = evaluator(iter(ts_it), iter(forecast_it))

print(agg_metrics)

/opt/conda/envs/clts2/lib/python3.10/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
[14:23:44] /work/mxnet/src/base.cc:79: cuDNN lib mismatch: linked-against version 8101 != compiled-against version 8500.  Set MXNET_CUDNN_LIB_CHECKING=0 to quiet this warning.
100%|██████████| 50/50 [00:02<00:00, 20.70it/s, epoch=1/1, avg_epoch_loss=3.69e+3]
Running evaluation: 414it [00:16, 25.80it/s]


{'MSE': 101894675.60529852, 'abs_error': 29009996.051239014, 'abs_target_sum': 145558863.59960938, 'abs_target_mean': 7324.822041043146, 'seasonal_error': 336.9046924038305, 'MASE': 12.371873980916131, 'MAPE': 0.8961260776588882, 'sMAPE': 0.40215173498060003, 'MSIS': 494.8749590281817, 'num_masked_target_values': 0.0, 'QuantileLoss[0.1]': 36398130.45469226, 'Coverage[0.1]': 0.5782004830917874, 'QuantileLoss[0.2]': 34551096.897538, 'Coverage[0.2]': 0.5782004830917874, 'QuantileLoss[0.3]': 32704063.340383716, 'Coverage[0.3]': 0.5782004830917874, 'QuantileLoss[0.4]': 30857029.783229448, 'Coverage[0.4]': 0.5782004830917874, 'QuantileLoss[0.5]': 29009996.226075172, 'Coverage[0.5]': 0.5782004830917874, 'QuantileLoss[0.6]': 27162962.668920897, 'Coverage[0.6]': 0.5782004830917874, 'QuantileLoss[0.7]': 25315929.111766625, 'Coverage[0.7]': 0.5782004830917874, 'QuantileLoss[0.8]': 23468895.554612346, 'Coverage[0.8]': 0.5782004830917874, 'QuantileLoss[0.9]': 21621861.997458074, 'Coverage[0.9]': 0.

In [4]:
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.mx.model.deepar import DeepAREstimator
from gluonts.mx.model.seq2seq import MQCNNEstimator
from gluonts.mx.model.wavenet import WaveNetEstimator
from gluonts.mx.trainer import Trainer
from gluonts.dataset.util import to_pandas
from gluonts.evaluation.backtest import make_evaluation_predictions
import numpy as np



# Load the "m4_hourly" dataset
dataset = get_dataset("m4_hourly")
train_entry = next(iter(dataset.train))
train_series = to_pandas(train_entry)

# Create a DeepAR estimator
estimator = DeepAREstimator(
    freq=train_series.index.freq.freqstr,
    prediction_length=48,
    trainer=Trainer(epochs=1, ctx='gpu(0)')
)
# estimator = MQCNNEstimator(
#     freq=train_series.index.freq.freqstr,
#     prediction_length=48,
#     context_length=
#     trainer=Trainer(epochs=1, ctx='cpu', hybridize=False)
# )

# Train the estimator on the training data
predictor = estimator.train(dataset.train)

# Use the predictor to make predictions
test_entry = next(iter(dataset.test))
test_series = to_pandas(test_entry)
# forecast = predictor.predict(test_series.index[0:48].values.reshape(1, -1))
forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_samples=100
)
forecast_list = list(forecast_it)
point_forecasts = []
for f in forecast_list:
    point_forecasts.append(np.mean(f.samples))

print(point_forecasts)

print(point_forecasts)

/opt/conda/envs/clts2/lib/python3.10/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
[14:19:41] /work/mxnet/src/base.cc:79: cuDNN lib mismatch: linked-against version 8101 != compiled-against version 8500.  Set MXNET_CUDNN_LIB_CHECKING=0 to quiet this warning.
100%|██████████| 50/50 [00:09<00:00,  5.06it/s, epoch=1/1, avg_epoch_loss=5.61]


[712.06976, 3099.666, 1295.6606, 5254.0474, 2758.132, 6739.0635, 41638.887, 32675.268, 38622.543, 460.45834, 33708.035, 9155.648, 617.13306, 182.92618, 15201.203, 1366.4558, 55573.094, 33641.332, 2557.0364, 7539.308, 17865.41, 691.948, 295.54846, 661.96643, 2514.2844, 19049.947, 2125.2705, 4148.7754, 5477.3154, 119475.96, 109065.0, 100502.625, 5663.445, 45603.254, 1368.237, 25294.047, 2050.2466, 2767.7705, 119734.34, 2081.7764, 3539.8167, 3946.435, 3743.1091, 1108.6484, 38104.48, 180.5606, 37146.715, 4879.6606, 16946.076, 38267.387, 427.66965, 2882.23, 2082.9812, 428.73773, 539.51263, 24100.334, 585534.4, 3440.4314, 1225.8464, 980.4658, 3348.35, 449.09, 7932.8125, 1801.3627, 12242.269, 31759.867, 32751.893, 38705.633, 934.28107, 26416.99, 9817.205, 260.66025, 432.19354, 510.5751, 15294.087, 687.97363, 610.6842, 55416.082, 31783.754, 2443.2603, 5563.8267, 16414.518, 995.9561, 525.613, 924.63666, 255.96924, 757.1403, 945.06055, 1527.7343, 16778.69, 1955.0406, 3153.1592, 5775.0386, 105481

In [12]:
target = data['target']
target[:-1]

array([ 15.,  13.,  13.,  16.,  26.,  51.,  91., 103., 106.,  89.,  81.,
        88.,  80., 112., 110., 102.,  94., 116., 110.,  59.,  57.,  50.,
        35.,  24.,  18.,  14.,  19.,  18.,  21.,  62.,  71.,  95., 101.,
       110., 103., 104., 112., 138., 119., 121., 157., 145., 122., 100.,
        90.,  61.,  56.,  29.,  26.,  20.,  16.,  17.,  24.,  31.,  44.,
        62.,  94., 135., 116., 146., 132., 122., 137., 160., 187., 218.,
       195., 158., 156.,  97.,  86.,  64.,  48.,  28.,  26.,  18.,  32.,
        32.,  54., 100., 150., 200., 330., 404., 470., 496., 480., 536.,
       548., 588., 560., 452., 334., 258., 112.,  54.,  18.,  14.,  15.,
        26.,  37.,  77., 105., 110., 119.,  90., 111., 118.,  99.,  86.,
        99.,  98.,  96.,  90.,  81.,  69.,  49.,  49.,  34.,  27.,  16.,
        15.,  11.,  19.,  28.,  60.,  99.,  93., 111., 113., 104.,  73.,
        73., 111.,  94.,  92., 103.,  79.,  77.,  64.,  57.,  56.,  36.,
        24.,  18.,  11.,  14.,  22.,  32.,  65.,  8

In [15]:
from gluonts.dataset.common import ListDataset
def difference(dataset):
    differenced_dataset = []
    for data in dataset:
        target = data['target']
        differenced_target = target[1:] - target[:-1]
        differenced_data = {**data, 'target': differenced_target}
        differenced_dataset.append(differenced_data)
    return differenced_dataset

train_dataset = difference(dataset.train)
test_dataset = difference(dataset.test)

In [17]:
train_dataset[0]

{'target': array([ -19.,    0.,  -27.,  -48.,  -68.,  -21.,  -27.,  -13.,  -12.,
          13.,   14.,   23.,   35.,   38.,   61.,   56.,   56.,   49.,
          40.,   23.,   16.,   12.,    2.,  -32.,  -53.,  -14.,  -49.,
         -75.,  -58.,  -65.,  -25.,  -24.,   -6.,   -3.,   16.,   21.,
          30.,   55.,   45.,   41.,   23.,  -22.,   -5.,    6.,   23.,
          32.,   14.,    4.,  -27.,    1.,  -29.,  -59.,  -52.,  -40.,
         -29.,   -5.,  -22.,    2.,   14.,   -3.,   34.,   56.,   68.,
          65.,   47.,   46.,   27.,   27.,   11.,    1.,  -19.,  -31.,
         -53.,  -20.,  -42.,  -48.,  -55.,  -40.,  -37.,  -23.,  -16.,
          -7.,    4.,   -3.,   42.,   64.,   75.,   51.,   43.,    2.,
          11.,   15.,    8.,   -4.,  -27.,  -63.,  -54.,  -29.,  -36.,
         -21.,  -41.,  -38.,  -32.,  -14.,  -11.,    3.,   -2.,    1.,
          33.,   45.,   29.,    7.,    0.,    7.,   11.,   14.,   16.,
          25.,   10.,  -19.,  -10.,   11.,  -31.,  -44.,  -51.,  -3

In [ ]:
import mxnet as mx

for i in range(mx.context.num_gpus()):
    free_mem, total_mem = mx.context.gpu_memory_info(i)
    print(f"GPU {i} memory: free={free_mem / (1024 * 1024)}MB, total={total_mem / (1024 * 1024)}MB")


